# Table 2 Recreation: MobileNet
The paper uses an unspecified version of MobileNetv3. I implemented MobileNetv3-Small that I used which was downloaded from GitHub, as well as  MobileNetv2 and MobileNetV3-Large which are both offered by pytorch. 

I also implement EfficientNet and DPN-92 which were both available on Github.

Summary: 

Test set | MobileNetV2 | MobileNetV3-Small | MobileNetV3-Large | EfficientNet | DPN-92
--- | --- | --- | --- | --- | --- 
original | 93.80% | 91.46% | 95.01% | 94.74% | 97.21%
only_bg_t | 13.16% | 14.44% | 16.00% | 10.57% | 17.58%
mixed_same | 80.05% | 74.42% | 81.68% | 87.11%| 90.47%
mixed_rand | 70.49% | 60.44% | 70.25% | 80.17% | 85.98%
BG-Gap | 9.56% | 13.98% | 11.43% | 6.94% | 4.49%

The main point that the paper is emphasising still holds event though there is slight discrepencies in most accuracy figures. Clearly these models also somewhat rely on the background signal.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip drive/MyDrive/ReproducabilityChallenge/only_bg_t.zip
!unzip drive/MyDrive/ReproducabilityChallenge/mixed_rand.zip
!unzip drive/MyDrive/ReproducabilityChallenge/original.zip
!unzip drive/MyDrive/ReproducabilityChallenge/mixed_same.zip

Streaming output truncated to the last 5000 lines.
  inflating: mixed_same/val/04_carnivore/fg_n02115641_41341_bg_n02233338_15470.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02115641_41341_bg_n02233338_15470.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02128757_04135_bg_n02279972_20458.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02128757_04135_bg_n02279972_20458.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02128757_22170_bg_n02264363_47939.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02128757_22170_bg_n02264363_47939.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02119022_00157_bg_n02177972_01737.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02119022_00157_bg_n02177972_01737.JPEG  
  inflating: mixed_same/val/04_carnivore/fg_n02119022_06668_bg_n02276258_45550.JPEG  
  inflating: __MACOSX/mixed_same/val/04_carnivore/._fg_n02119022_06668_bg_n02276258_45550.JPEG  
  inflating: mixed_same/val/04_car

In [3]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

     |████████████████████████████████| 143kB 17.4MB/s 


In [19]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchbearer import Trial
from torchvision.datasets import ImageFolder
import os
import json
import torchvision
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


!unzip mixed_same.zip

In [5]:
def getDataLoader(datadir = 'mixed_same/val'):
    # convert each image to tensor format
    preprocess = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    #datadir = '/Volumes/DRIVE/test_sets/mixed_same/val'
    #datadir = os.path.join('/Volumes/DRIVE/test_sets/mixed_same/', 'val')
    imgFolder = ImageFolder(datadir, transform=preprocess)
    testloader = DataLoader(imgFolder, batch_size=32, shuffle=False)
    return testloader

In [17]:
def getModelAccuracy(model, testloader, in_to_in9):
    # Compute the model accuracy on the test set
    correct = 0
    total = 0

    for inputs, labels in testloader:
        outputs = model(inputs)

        for (output, real_label) in zip(outputs, labels):
            model_output = in_to_in9[str(torch.argmax(output).item())]

            if model_output == real_label.item():
                correct += 1
            total += 1


    print('Test Accuracy: %2.2f %%' % ((100.0 * correct) / total))

def getModelAccuracyCuda(model, testloader, in_to_in9):
    # Compute the model accuracy on the test set
    correct = 0
    total = 0

    for inputs, labels in testloader:
        outputs = model(inputs.to('cuda:0'))

        for (output, real_label) in zip(outputs, labels):
            model_output = in_to_in9[str(torch.argmax(output).item())]

            if model_output == real_label.item():
                correct += 1
            total += 1


    print('Test Accuracy: %2.2f %%' % ((100.0 * correct) / total))


In [7]:
def getIn2In9(root = ''):
    with open(root + 'in_to_in9.json') as f:
      in_to_in9 = json.load(f)
    return in_to_in9

In [9]:
torchvision.__version__

'0.9.0+cu101'

# MobileNetV2
Torch ffers mobilenetv2, whereas the paper in question uses mobilenetv3

In [8]:
#Import the imagenet pretrained mobilenetv2 model
mobilenet = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
mobilenet.to('cuda:0')
mobilenet.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [9]:
#Get the mixed_same dataset
mixed_same = getDataLoader(datadir = 'mixed_same/val')

In [13]:
#Get the imagenet to in9l class converter
in_to_in9 = getIn2In9(root='drive/MyDrive/ReproducabilityChallenge/')

In [21]:
#Compute model accuracy on mixed_same
getModelAccuracyCuda(mobilenet, mixed_same, in_to_in9)

Test Accuracy: 80.05 %


In [22]:
#Get the mixed_rand dataset
mixed_rand = getDataLoader(datadir = 'mixed_rand/val')

In [23]:
#Compute model accuracy on mixed_same
getModelAccuracyCuda(mobilenet, mixed_rand, in_to_in9)

Test Accuracy: 70.49 %


Using mobilenetv2 we get a test accuracy of 80.05% on mixed_same and 70.49% on mixed_rand giving a BG-GAP of 9.56%

In [24]:
#Get the original dataset
original = getDataLoader(datadir = 'original/val')

In [25]:
#Compute model accuracy on original
getModelAccuracyCuda(mobilenet, original, in_to_in9)

Test Accuracy: 93.80 %


In [26]:
#Get the only_bg_t dataset
only_bg_t = getDataLoader(datadir = 'only_bg_t/val')

In [27]:
#Compute model accuracy on only_bg_t
getModelAccuracyCuda(mobilenet, only_bg_t, in_to_in9)

Test Accuracy: 13.16 %


# MobileNetV3-Small
This is made available on github
Credit: https://github.com/kuan-wang/pytorch-mobilenet-v3

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F


__all__ = ['MobileNetV3', 'mobilenetv3']


def conv_bn(inp, oup, stride, conv_layer=nn.Conv2d, norm_layer=nn.BatchNorm2d, nlin_layer=nn.ReLU):
    return nn.Sequential(
        conv_layer(inp, oup, 3, stride, 1, bias=False),
        norm_layer(oup),
        nlin_layer(inplace=True)
    )


def conv_1x1_bn(inp, oup, conv_layer=nn.Conv2d, norm_layer=nn.BatchNorm2d, nlin_layer=nn.ReLU):
    return nn.Sequential(
        conv_layer(inp, oup, 1, 1, 0, bias=False),
        norm_layer(oup),
        nlin_layer(inplace=True)
    )


class Hswish(nn.Module):
    def __init__(self, inplace=True):
        super(Hswish, self).__init__()
        self.inplace = inplace

    def forward(self, x):
        return x * F.relu6(x + 3., inplace=self.inplace) / 6.


class Hsigmoid(nn.Module):
    def __init__(self, inplace=True):
        super(Hsigmoid, self).__init__()
        self.inplace = inplace

    def forward(self, x):
        return F.relu6(x + 3., inplace=self.inplace) / 6.


class SEModule(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            Hsigmoid()
            # nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


class Identity(nn.Module):
    def __init__(self, channel):
        super(Identity, self).__init__()

    def forward(self, x):
        return x


def make_divisible(x, divisible_by=8):
    import numpy as np
    return int(np.ceil(x * 1. / divisible_by) * divisible_by)


class MobileBottleneck(nn.Module):
    def __init__(self, inp, oup, kernel, stride, exp, se=False, nl='RE'):
        super(MobileBottleneck, self).__init__()
        assert stride in [1, 2]
        assert kernel in [3, 5]
        padding = (kernel - 1) // 2
        self.use_res_connect = stride == 1 and inp == oup

        conv_layer = nn.Conv2d
        norm_layer = nn.BatchNorm2d
        if nl == 'RE':
            nlin_layer = nn.ReLU # or ReLU6
        elif nl == 'HS':
            nlin_layer = Hswish
        else:
            raise NotImplementedError
        if se:
            SELayer = SEModule
        else:
            SELayer = Identity

        self.conv = nn.Sequential(
            # pw
            conv_layer(inp, exp, 1, 1, 0, bias=False),
            norm_layer(exp),
            nlin_layer(inplace=True),
            # dw
            conv_layer(exp, exp, kernel, stride, padding, groups=exp, bias=False),
            norm_layer(exp),
            SELayer(exp),
            nlin_layer(inplace=True),
            # pw-linear
            conv_layer(exp, oup, 1, 1, 0, bias=False),
            norm_layer(oup),
        )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV3(nn.Module):
    def __init__(self, n_class=1000, input_size=224, dropout=0.8, mode='small', width_mult=1.0):
        super(MobileNetV3, self).__init__()
        input_channel = 16
        last_channel = 1280
        if mode == 'large':
            # refer to Table 1 in paper
            mobile_setting = [
                # k, exp, c,  se,     nl,  s,
                [3, 16,  16,  False, 'RE', 1],
                [3, 64,  24,  False, 'RE', 2],
                [3, 72,  24,  False, 'RE', 1],
                [5, 72,  40,  True,  'RE', 2],
                [5, 120, 40,  True,  'RE', 1],
                [5, 120, 40,  True,  'RE', 1],
                [3, 240, 80,  False, 'HS', 2],
                [3, 200, 80,  False, 'HS', 1],
                [3, 184, 80,  False, 'HS', 1],
                [3, 184, 80,  False, 'HS', 1],
                [3, 480, 112, True,  'HS', 1],
                [3, 672, 112, True,  'HS', 1],
                [5, 672, 160, True,  'HS', 2],
                [5, 960, 160, True,  'HS', 1],
                [5, 960, 160, True,  'HS', 1],
            ]
        elif mode == 'small':
            # refer to Table 2 in paper
            mobile_setting = [
                # k, exp, c,  se,     nl,  s,
                [3, 16,  16,  True,  'RE', 2],
                [3, 72,  24,  False, 'RE', 2],
                [3, 88,  24,  False, 'RE', 1],
                [5, 96,  40,  True,  'HS', 2],
                [5, 240, 40,  True,  'HS', 1],
                [5, 240, 40,  True,  'HS', 1],
                [5, 120, 48,  True,  'HS', 1],
                [5, 144, 48,  True,  'HS', 1],
                [5, 288, 96,  True,  'HS', 2],
                [5, 576, 96,  True,  'HS', 1],
                [5, 576, 96,  True,  'HS', 1],
            ]
        else:
            raise NotImplementedError

        # building first layer
        assert input_size % 32 == 0
        last_channel = make_divisible(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2, nlin_layer=Hswish)]
        self.classifier = []

        # building mobile blocks
        for k, exp, c, se, nl, s in mobile_setting:
            output_channel = make_divisible(c * width_mult)
            exp_channel = make_divisible(exp * width_mult)
            self.features.append(MobileBottleneck(input_channel, output_channel, k, s, exp_channel, se, nl))
            input_channel = output_channel

        # building last several layers
        if mode == 'large':
            last_conv = make_divisible(960 * width_mult)
            self.features.append(conv_1x1_bn(input_channel, last_conv, nlin_layer=Hswish))
            self.features.append(nn.AdaptiveAvgPool2d(1))
            self.features.append(nn.Conv2d(last_conv, last_channel, 1, 1, 0))
            self.features.append(Hswish(inplace=True))
        elif mode == 'small':
            last_conv = make_divisible(576 * width_mult)
            self.features.append(conv_1x1_bn(input_channel, last_conv, nlin_layer=Hswish))
            # self.features.append(SEModule(last_conv))  # refer to paper Table2, but I think this is a mistake
            self.features.append(nn.AdaptiveAvgPool2d(1))
            self.features.append(nn.Conv2d(last_conv, last_channel, 1, 1, 0))
            self.features.append(Hswish(inplace=True))
        else:
            raise NotImplementedError

        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),    # refer to paper section 6
            nn.Linear(last_channel, n_class),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.mean(3).mean(2)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)





In [29]:
mobilenetv3 = MobileNetV3()
mobilenetv3.load_state_dict(torch.load('mobilenetv3_small_67.4.pth.tar', map_location='cpu'))
mobilenetv3.to('cuda:0')
mobilenetv3 = mobilenetv3.eval() 

FileNotFoundError: ignored

In [ ]:
#Compute model accuracy on mixed_same
getModelAccuracy(mobilenetv3, mixed_same, in_to_in9)

Test Accuracy: 74.42 %


In [ ]:
#Compute model accuracy on mixed_rand
getModelAccuracy(mobilenetv3, mixed_rand, in_to_in9)

Test Accuracy: 60.44 %


In [ ]:
#Compute model accuracy on original
getModelAccuracy(mobilenetv3, original, in_to_in9)

Test Accuracy: 91.46 %


In [ ]:
#Compute model accuracy on only_bg_t
getModelAccuracy(mobilenetv3, only_bg_t, in_to_in9)

Test Accuracy: 14.44 %


# MobileNetV3-Large: Pytorch Implementation

In [30]:
mv3 = torchvision.models.mobilenet_v3_large(pretrained=True)
mv3.to('cuda:0')
mv3.eval()
#mv3 = torch.hub.load('pytorch/vision:v0.9.0', 'mobilenet_v3_large', pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Identity()
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=

In [31]:
#Compute model accuracy on mixed_same
getModelAccuracyCuda(mv3, mixed_same, in_to_in9)

Test Accuracy: 81.68 %


In [32]:
#Compute model accuracy on mixed_rand
getModelAccuracyCuda(mv3, mixed_rand, in_to_in9)

Test Accuracy: 70.25 %


In [33]:
#Compute model accuracy on original
getModelAccuracyCuda(mv3, original, in_to_in9)

Test Accuracy: 95.01 %


In [34]:
#Compute model accuracy on only_bg_t
getModelAccuracyCuda(mv3, only_bg_t, in_to_in9)

Test Accuracy: 16.00 %


# EfficientNet-b0
Model provided by https://github.com/lukemelas/EfficientNet-PyTorch#usage

In [35]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=e3da2f956b071aba13d750333d742fa0b95942224bbfefd2b057406803edb9f1
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [37]:
from efficientnet_pytorch import EfficientNet
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
efficientnet.to('cuda:0')
efficientnet.eval()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [38]:
getModelAccuracyCuda(efficientnet, mixed_same, in_to_in9)

Test Accuracy: 87.11 %


In [39]:
#Compute model accuracy on mixed_rand
getModelAccuracyCuda(efficientnet, mixed_rand, in_to_in9)

Test Accuracy: 80.17 %


In [40]:
#Compute model accuracy on original
getModelAccuracyCuda(efficientnet, original, in_to_in9)

Test Accuracy: 94.74 %


In [41]:
#Compute model accuracy on only_bg_t
getModelAccuracyCuda(efficientnet, only_bg_t, in_to_in9)

Test Accuracy: 10.57 %


# DPN-92
Model provided by https://github.com/rwightman/pytorch-dpn-pretrained

In [44]:
dpn = torch.hub.load('rwightman/pytorch-dpn-pretrained', 'dpn92', pretrained=True)
dpn.to('cuda:0')
dpn.eval()

Using cache found in /root/.cache/torch/hub/rwightman_pytorch-dpn-pretrained_master


DPN(
  (features): Sequential(
    (conv1_1): InputBlock(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (conv2_1): DualPathBlock(
      (c1x1_w_s1): BnActConv2d(
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (conv): Conv2d(64, 288, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (c1x1_a): BnActConv2d(
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (conv): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (c3x3_b): BnActConv2d(
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True, track_running_s

In [45]:
getModelAccuracyCuda(dpn, mixed_same, in_to_in9)

Test Accuracy: 90.47 %


In [46]:
getModelAccuracyCuda(dpn, mixed_rand, in_to_in9)

Test Accuracy: 85.98 %


In [47]:
getModelAccuracyCuda(dpn, original, in_to_in9)

Test Accuracy: 97.21 %


In [48]:
getModelAccuracyCuda(dpn, only_bg_t, in_to_in9)

Test Accuracy: 17.58 %
